In [2]:
# Importing the required packages
import base64
import requests
import pandas as pd
import toml
import json

In [3]:
# Reading the output from '02_Get_repo_names.jl'
df = pd.read_csv("./Julia_Packages_List_Repo.csv")

In [4]:
# Converting the names and URLs to a list
repo_urls = df.RepoURL.tolist()
package_names = df.PackageName.tolist()

In [5]:
# Function to remove trailing .git from the reponame if it exists

def clean_reponame(reponame):
    
    if reponame.endswith(".git"):
        reponame = reponame[:-4]
    return reponame

In [6]:
# Iterating over each package name and URL to extract metadata and store it to results list

results = []
for r, p in zip(repo_urls, package_names):
    user = r.split("/")[-2]
    repo_name = clean_reponame(r.split("/")[-1])
    path_to_file = "Project.toml"
    url = "https://raw.githubusercontent.com/{user}/{repo_name}/master/Project.toml".format(user=user, repo_name=repo_name)
    print(url)
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to get Project.toml for {repo_name}".format(repo_name=repo_name))
        continue
    toml_content = response.content.decode("utf-8")
    toml_data = toml.loads(toml_content)
    deps_json = json.dumps(toml_data)
    results.append([p, r, deps_json])

https://raw.githubusercontent.com/pfitzseb/REPLTreeViews.jl/master/Project.toml
https://raw.githubusercontent.com/TuringLang/TuringGLM.jl/master/Project.toml
https://raw.githubusercontent.com/francescoalemanno/BayesHistogram.jl/master/Project.toml
https://raw.githubusercontent.com/JuliaBinaryWrappers/KCP_jll.jl/master/Project.toml
https://raw.githubusercontent.com/eschnett/ProgressView.jl/master/Project.toml
https://raw.githubusercontent.com/JuliaBinaryWrappers/oneAPI_Level_Zero_jll.jl/master/Project.toml
https://raw.githubusercontent.com/OpenMendel/OrdinalGWAS.jl/master/Project.toml
https://raw.githubusercontent.com/LiScI-Lab/HeartRateVariability.jl/master/Project.toml
https://raw.githubusercontent.com/JuliaBinaryWrappers/CUTENSOR_CUDA110_jll.jl/master/Project.toml
https://raw.githubusercontent.com/JuliaBinaryWrappers/gb_jll.jl/master/Project.toml
https://raw.githubusercontent.com/BioJulia/BioAlignments.jl/master/Project.toml
https://raw.githubusercontent.com/SeismicJulia/SeisPlot.jl/

In [7]:
# Creating a dataframe to store results list in a structured format
df_results = pd.DataFrame(results, columns=["Package", "RepoURL", "Metadata"])

In [8]:
# Saving the dataframe to .csv format
df_results.to_csv("./Julia_Packages_List_Repo_Deps.csv", index=False)